# Autonomous Line Following Using JetBot
` Inspired by and based on: RoboCupJunior Rescue Line – Rules 2018`

An autonomous robot should follow a black line while overcoming different problems in a
modular Field formed by tiles with different patterns. The floor is white in color and the tiles are on
different levels connected with ramps. The end of the field will be marked with a strip of reflective silver
tape on the floor. Teams are not allowed to give their robot any advance information about the field as the
robot is supposed to recognize the field by itself

### Import Libraries

In [1]:
from jetbot import Robot
import cv2
import numpy as np

from jetbot import Robot, Camera, bgr8_to_jpeg
import threading
import traitlets

import os
import json
import glob
import datetime
import time
import asyncio
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

## Create jetbot and camera instance

Now that we've imported the ``Robot`` class we can initialize the class *instance* as follows. 

In [2]:
##create robot instance
robot = Robot()

#creating camera instance 
camera = Camera()
print('Camera has been Initialized successfully!')

Camera has been Initialized successfully!


## Line following code

In [3]:
import cv2
import numpy as np
import ipywidgets as widgets
from IPython.display import display
x, y, w, h = 50, 50, 100, 100

turn_time = time.time() # current time 

# Create display widgets
image_widget = widgets.Image(format='jpeg', width=224, height=224)
gray_widget = widgets.Image(format='jpeg', width=224, height=224)
green_widget = widgets.Image(format='jpeg', width=224, height=224)
red_widget = widgets.Image(format='jpeg', width=224, height=224)
cx_label = widgets.Label(value='cx: ')
cy_label = widgets.Label(value='cy: ')
average_x_label = widgets.Label(value='Avg x: ')
average_y_label = widgets.Label(value='Avg y: ')
area_label = widgets.Label(value= 'Area: ')

# Display the widgets
display(image_widget, gray_widget, green_widget,red_widget,cx_label,cy_label,average_x_label, average_y_label, area_label)


# Function to convert BGR image to JPEG format
def bgr8_to_jpeg(value):
    return bytes(cv2.imencode('.jpg', value)[1])

def filter_outliers(intersections, m=0.5):
    if not intersections:
        return []

    # Convert the list of tuples to a numpy array for easier manipulation
    intersection_points = np.array(intersections)
    
    # Calculate the mean and standard deviation of the points
    mean = np.mean(intersection_points, axis=0)
    std_dev = np.std(intersection_points, axis=0)
    
    # Calculate the z-score for each point (distance from the mean in terms of standard deviation)
    z_scores = (intersection_points - mean) / std_dev
    
    # Filter the points where the absolute z-score is less than 'm' for both x and y
    filtered_points = intersection_points[(np.abs(z_scores) < m).all(axis=1)]
    
    return filtered_points


#function to find the intersection of the line 
def find_intersection(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Manually set the threshold value
    manual_threshold_value = 70
    # Apply manual thresholding
    _, thresholded = cv2.threshold(gray, manual_threshold_value, 255, cv2.THRESH_BINARY_INV)
    canny_image=cv2.Canny(thresholded, 50, 150)
    lines=cv2.HoughLinesP(canny_image, rho=1, theta=np.pi/180, threshold=45, minLineLength=10, maxLineGap=300)
    
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(image, (x1, y1), (x2, y2),  (255, 0, 0), 1)
    intersections = []
    average_intersection = None
    if lines is not None:
        for i, line1 in enumerate(lines):
            for line2 in lines[i+1:]:
                x1, y1, x2, y2 = line1[0]
                x3, y3, x4, y4 = line2[0]

                # Line AB represented as a1x + b1y = c1
                a1 = y2 - y1
                b1 = x1 - x2
                c1 = a1*x1 + b1*y1

                # Line CD represented as a2x + b2y = c2
                a2 = y4 - y3
                b2 = x3 - x4
                c2 = a2*x3 + b2*y3

                determinant = a1*b2 - a2*b1

                if determinant != 0:
                    x = (b2*c1 - b1*c2) / determinant
                    y = (a1*c2 - a2*c1) / determinant
                    
                    intersections.append((int(x), int(y)))
                    
        intersections = filter_outliers(intersections)
        
        if len(intersections) > 0: 
            average_x = sum(x for x, y in intersections) / len(intersections)
            average_y = sum(y for x, y in intersections) / len(intersections)
            average_intersection = (int(average_x), int(average_y))
    
    return average_intersection

# Function to process the image and perform red stop detection
def process_stop_sign(image,cropped_image):
    # Convert BGR to HSV
    hsv_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2HSV)
    
    #ranges for the red color
    lower_red1 = np.array([0, 100, 100])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([160, 100, 100])
    upper_red2 = np.array([179, 255, 255])
    
    # Create masks for the red color ranges
    mask_red1 = cv2.inRange(hsv_image, lower_red1, upper_red1)
    mask_red2 = cv2.inRange(hsv_image, lower_red2, upper_red2)
    
    # Combine the masks for the two red ranges
    mask_red = cv2.add(mask_red1, mask_red2)
    # Find contours for red square
    contours_red, _ = cv2.findContours(mask_red, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    if (np.array(contours_red, dtype=object).size < 3): ## size 3-5
         line_follow(image)
    else:
        #find the largest contour
        largest_contour_red = max(contours_red, key=cv2.contourArea)
        M = cv2.moments(largest_contour_red)
        if M["m00"] != 0:
            cx = int(M["m10"] / M["m00"])
            cy = int(M["m01"] / M["m00"])
            cv2.circle(image, (cx, cy), 5, (255, 0, 0), -1)
            #if you are very close, stop the robot
            if cy > 85:
            #stop the robot as you are at stop condition 
                robot.stop()

    red_widget.value = bgr8_to_jpeg(mask_red)
    
    
# Function to process the image and perform green square detection
def process_green_square(image,cropped_image):
    global turn_time

    # Convert BGR to HSV
    hsv = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2HSV)

    # Define the lower and upper bounds for green color in HSV
    lower_green = np.array([35, 100, 25])
    upper_green = np.array([90, 255, 255])

    # Threshold the image to get only green regions
    mask_green = cv2.inRange(hsv, lower_green, upper_green)

    # Find contours for green square
    contours_green, _ = cv2.findContours(mask_green, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    if (np.array(contours_green, dtype=object).size < 3): ## size 3-5
        process_stop_sign(image,cropped_image)
    else:
        #find the largest contour 
        largest_contour_green = max(contours_green, key=cv2.contourArea)
        # getting the area for the U turn 
        total_area=sum(cv2.contourArea(contour) for contour in contours_green)
        area_label.value = f'Area: {total_area}'
        
        M_green = cv2.moments(largest_contour_green)
        if M_green["m00"] != 0:
            cx_green = int(M_green["m10"] / M_green["m00"])
            cy_green = int(M_green["m01"] / M_green["m00"])
            
            cx_label.value = f'cx: {cx_green}'
            cy_label.value = f'cy: {cy_green}'
            
            cv2.circle(cropped_image, (cx_green, cy_green), 5, (0, 255, 0), -1)
            average_x=0
            average_y=0
            intersection=find_intersection(cropped_image)
            if intersection is not None: 
                average_x, average_y=intersection
                cv2.circle(cropped_image, (average_x, average_y), 5, (0, 0, 255), -1)
                
            average_x_label.value = f'Avg x: {average_x}'
            average_y_label.value = f'Avg y: {average_y}'
            
            #check the turn conditions and make a turn a decision 
            if (time.time() - turn_time > 0.5) and cy_green> 30 and (intersection is not None) and total_area < 1100: # works for other case 
                if cx_green > average_x :
                    robot.forward(0.1)
                    time.sleep(0.4)
                    robot.right(0.11)
                    time.sleep(0.6)
                    robot.forward(0.15)
                    time.sleep(0.2)
                    turn_time =  time.time()
                else:
                    robot.forward(0.1)
                    time.sleep(0.4)
                    robot.left(0.095)
                    time.sleep(0.6)
                    robot.stop()
                    turn_time =  time.time()
                    
            # Make a U-turn case 
            elif (time.time() - turn_time > 0.5) and cy_green < 20 and total_area > 1100: # CONFIRM IN THE MORNING theses values -> lighing might impact 
                robot.forward(0.1)
                time.sleep(0.4)
                robot.left(0.11)
                time.sleep(2.2)
                turn_time =  time.time()
                
            else:
                robot.forward(0.09)
                time.sleep(0.00001) # to consider 
                    
    green_widget.value = bgr8_to_jpeg(cropped_image)

# Line following logic
def line_follow(image):
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Manually set the threshold value
    manual_threshold_value = 70

    # Apply manual thresholding
    _, thresholded = cv2.threshold(gray, manual_threshold_value, 255, cv2.THRESH_BINARY_INV)

    # Find contours for line following
    contours, _ = cv2.findContours(thresholded, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        #find the largest contour 
        largest_contour = max(contours, key=cv2.contourArea)
        M = cv2.moments(largest_contour)
        if M["m00"] != 0:
            cx = int(M["m10"] / M["m00"])
            cy = int(M["m01"] / M["m00"])
            
            cv2.circle(image, (cx, cy), 5, (255, 0, 0), -1)
            #make a line follow decision 
            if cx > 50 and cx < 100:
                robot.forward(0.12)
            elif cx >= 100:
                robot.right(0.1)
            elif cx <= 50:
                robot.left(0.1)
            elif cy >150:
                robot.forward(0.12)
    else:
        #if you do not see any black liene contour
        robot.forward(0.12)

    gray_widget.value = bgr8_to_jpeg(thresholded)
    image_widget.value = bgr8_to_jpeg(image)

# Function to update the image widget
def update_image(change):
    #print("Updating image now")
    image = change['new']
    
    #crop the image for line following 
    top_left = (47, 90)  # Example coordinates, adjust as needed
    bottom_right = (207, 224) 
    w = bottom_right[0] - top_left[0]
    h = bottom_right[1] - top_left[1]
    cropped_image = image[top_left[1]:top_left[1] + h, top_left[0]:top_left[0] + w]
    
    #crop the image for green detection and red stop sign 
    top_left_main = (39,30)
    bottom_right_main = (194,210)
    w_main = bottom_right_main[0] - top_left_main[0]
    h_main = bottom_right_main[1] - top_left_main[1]
    main_image = image[top_left_main[1]:top_left_main[1] + h_main, top_left_main[0]:top_left_main[0] + w_main]
    
    process_green_square(main_image,cropped_image)


# Attach the update function to the camera
camera.observe(update_image, names='value')



Image(value=b'', format='jpeg', height='224', width='224')

Image(value=b'', format='jpeg', height='224', width='224')

Image(value=b'', format='jpeg', height='224', width='224')

Image(value=b'', format='jpeg', height='224', width='224')

Label(value='cx: ')

Label(value='cy: ')

Label(value='Avg x: ')

Label(value='Avg y: ')

Label(value='Area: ')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: RuntimeWarning: invalid value encountered in true_divide


## Stop the camera observe process and stop running the robot 

In [ ]:
camera.unobserve(update_image, names='value')

time.sleep(4.0)  # add a small sleep to make sure frames have finished processing

robot.stop()



## References

* https://towardsdatascience.com/getting-started-in-ai-and-computer-vision-with-nvidia-jetson-nano-df2cacbd291c
* https://www.inspiritai.com/blogs/ai-blog/2021/8/31/student-blog-project-bridging-ai-and-robotics
* https://github.com/thehapyone/Platooning-Robot/blob/master/Robot/Main/Lane%20Detection/lane_detect1.py
* https://const-toporov.medium.com/line-following-robot-with-opencv-and-contour-based-approach-417b90f2c298
* https://rubikscode.net/2022/06/13/thresholding-edge-contour-and-line-detection-with-opencv/
* https://resources.altium.com/p/lane-recognition-and-tracking-nvidia-jetson-nano
* https://github.com/chrisdalke/ros-line-follower-robot
    